In [9]:
import numpy as np
import pandas as pd
import scipy.stats as sps

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

import matplotlib.pyplot as plt
import seaborn as sns

red = '#FF3300'
blue = '#0099CC'
green = '#00CC66'

from statsmodels.sandbox.stats.multicomp import multipletests
import tqdm
from collections import Counter
from statsmodels.stats.diagnostic import lilliefors

%matplotlib inline



# Задача  Оптимизационная
Физики из города Долгопрудный купили сверхточный термометр и датчик давления для исследования разреженных газов. В сосуд с изменяемым объемом они закачали некоторое количество одноатомного газа, причем он оказался достаточно разреженным. Затем они изменяли температуру данного сосуда и измеряли давление, причём теплоемкость газа в данном процессе можно считать постоянной. Значения, полученные в ходе эксперимента, они записывали в этот [файл](https://drive.google.com/file/d/1vg3fFktL01uqXNjUFRxlDdYWOyHOjDJL/view?usp=sharing). Постройте оценку теплоёмкости данного газа с наименьшей возможной дисперсией/погрешностью. 


*Указание 1: для процессов с постоянной теплоёмкостью зависимость давления от температуры имеет вид $P \propto T^{\alpha}$, где $\alpha$ &mdash; любое число.*

*Указание 2: Может быть полезным для уменьшения погрешности усреднять давление в некотором диапазоне температур.*



**Идея задачи:**
найдем коэффициент $\alpha$ из зависимости $P \propto T^{\alpha}$, ведь через него выражается теплоемкостью

Предварительную обработку данных делать не будем. После подсчетов посмотрим на погрешности и поймём, можно ли было так делать. Воспользуемся моделью линейной регрессии для анализа этой задачи.

В файле уберем пробел между названием величины  и ее единицами измерения, чтобы можно было сделать так:

In [12]:
data_1 = pd.read_csv("task1.txt", sep=" ") 
data_1.head()

,"T,a.u","P,a.u."
0,200.05,100
1,200.10,112
2,200.15,98
3,200.20,90
4,200.25,108


In [13]:
data_1.columns

Index(['T,a.u', 'P,a.u.'], dtype='object')

Произведем усреднение по температуре в пределах 10 градусов.

In [14]:
data_sorted = data_1.sort_values(by='T,a.u').to_numpy()


In [15]:
data_sorted = data_1.sort_values(by='T,a.u').to_numpy()
i=0
j=0
while i != len(data_sorted)-1:
    mean = []
    while data_sorted[: , 0][j] - data_sorted[: , 0][i] <5:
        mean.append( data_sorted[: , 1][j])
        j += 1
        if j >= len(data_sorted):
            break
    mean = np.mean(mean)
    data_sorted[:, 1][i:j] = mean
    i = j
    if j >= len(data_sorted):
            break
            
mean_data = pd.DataFrame()
mean_data['P,a.u.'] = data_sorted[:, 1][:]

mean_data['T,a.u'] = data_sorted[:, 0][:]
mean_data.head()

,"P,a.u.","T,a.u"
0,105.07,200.05
1,105.07,200.10
2,105.07,200.15
3,105.07,200.20
4,105.07,200.25


Посмотрим, сколько у нас уникальных давлений.

In [16]:
mean_data['P,a.u.'].unique().shape

(100,)

Так как погрешности распределены нормально(факт из общей физики), заменим все значения, соответствующие одинаковым давлениям, на средние:

In [17]:
upd_data = pd.DataFrame()
t =[]
for item in mean_data['P,a.u.'].unique():
    t.append(mean_data[mean_data['P,a.u.'] == item]['T,a.u'].mean())
    
upd_data['P,a.u.'] = mean_data['P,a.u.'].unique()
upd_data['T,a.u'] = t

Создадим линейный регрессор и обучим его на выборке предварительно разбив её на обучающую и тестовую в соотношении 80:20.

In [18]:
model = LinearRegression(fit_intercept=True)
x, y = np.log(upd_data['T,a.u'].to_numpy()), np.log(upd_data['P,a.u.'].to_numpy())
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)
model = model.fit(x_train.reshape(-1,1), y_train)

In [19]:
y_pred = model.predict(x_test.reshape(-1,1))

In [20]:
print(f"MSE:  {metrics.mean_absolute_error(y_test, y_pred) :.4}" + 
     "\n" + f"MAE:  {metrics.mean_squared_error(y_test, y_pred) :.4}" +
      '\n' + f"r2_score:  {metrics.r2_score(y_test, y_pred) :.5}"
     )

MSE:  0.004286
MAE:  3.105e-05
r2_score:  0.99994


In [21]:
alpha = model.coef_
print(f"Коэффицент:  \n {model.coef_[0] :}" +  "\n" + f"Свободный член: \n {model.intercept_ :.4}")

Коэффицент:  
 1.9966845826838284
Свободный член: 
 -5.97


In [22]:
n = (alpha[0])/(alpha[0]-1)
R = 8.31
c= 3/2 *R - R/(n-1)  
c

4.182551117897386

**Вывод**

Теплоемкость можно оценить так: 
c = (4.163 $\pm$ 0.004)

# Задача 2
В сфере статистики экзопланет ныне бытует мнение, что чем выше металличность звезды, тем выше вероятность того, что у нее будет планета гигант. Рассмотрим [часть выгрузки](https://drive.google.com/file/d/178IlxQdQYb8s5Fk9iWCW2zfBHaZnq-JJ/view?usp=sharing) базы данных экзопланетного архива NASA и проверим, так ли это. Конечно, в реальном исследовании стоит учитывать неоднородность данных, но для начала нам будет достаточно и сырых данных.

Проверьте гипотезу с использованием критерия согласия хи-квадрат. Для разбиения на корзины будем считать, что планета гигант &mdash; это планета с массой больше 0.3 масс Юпитера, а звезды для удобства разделим по металличности на меньше и больше 0. Так у нас получится 4 корзины. 



**Первая часть**

приведем данные в порядок

In [23]:
data = pd.read_csv("task2.csv") 

In [24]:
data.head()

,pl_name,hostname,discoverymethod,pl_bmassj,st_spectype,st_met,st_metratio,If planet mass>0.3 1; else,If metal>0 1; else 0,Unnamed: 9,Unnamed: 10
0,61 Vir b,61 Vir,Radial Velocity,0.0,1600.0,G5 V,0.0,10,[Fe/H],0.0,0.0
1,61 Vir c,61 Vir,Radial Velocity,0.0,5700.0,G5 V,0.0,10,[Fe/H],0.0,0.0
2,61 Vir d,61 Vir,Radial Velocity,0.0,7200.0,G5 V,0.0,10,[Fe/H],0.0,0.0
3,24 Sex b,24 Sex,Radial Velocity,1.0,99000.0,NaN,0.0,30,[Fe/H],1.0,0.0
4,24 Sex c,24 Sex,Radial Velocity,0.0,86000.0,NaN,0.0,30,[Fe/H],1.0,0.0


Сразу видно, что в табличке съехали названия столбцов, надо это исправить.

In [25]:
data.columns

Index(['pl_name', 'hostname', 'discoverymethod', 'pl_bmassj', 'st_spectype',
       'st_met', 'st_metratio', 'If planet mass>0.3 1; else',
       'If metal>0 1; else 0', 'Unnamed: 9', 'Unnamed: 10'],
      dtype='object')

In [26]:
columns_new = ['pl_name', 'hostname', 'discoverymethod', 'pl_bmassj_int', 'pl_bmassj_frac', 'st_spectype',
       'st_met_int', 'st_met_frac', 'st_metratio', 'If planet mass>0.3 1; else',
       'If metal>0 1; else 0']

In [27]:
data.columns = columns_new

In [28]:
data.columns

Index(['pl_name', 'hostname', 'discoverymethod', 'pl_bmassj_int',
       'pl_bmassj_frac', 'st_spectype', 'st_met_int', 'st_met_frac',
       'st_metratio', 'If planet mass>0.3 1; else', 'If metal>0 1; else 0'],
      dtype='object')

Теперь все названия удовлетворяют истинному содержанию колонок. Далее почистим те колонки, которые будем использовать, от значений 'NaN', потому что с ними невозможно работать.

In [29]:
data = data.dropna(subset=['If planet mass>0.3 1; else', 'If metal>0 1; else 0'])
data.head()

,pl_name,hostname,discoverymethod,pl_bmassj_int,pl_bmassj_frac,st_spectype,st_met_int,st_met_frac,st_metratio,If planet mass>0.3 1; else,If metal>0 1; else 0
0,61 Vir b,61 Vir,Radial Velocity,0.0,1600.0,G5 V,0.0,10,[Fe/H],0.0,0.0
1,61 Vir c,61 Vir,Radial Velocity,0.0,5700.0,G5 V,0.0,10,[Fe/H],0.0,0.0
2,61 Vir d,61 Vir,Radial Velocity,0.0,7200.0,G5 V,0.0,10,[Fe/H],0.0,0.0
3,24 Sex b,24 Sex,Radial Velocity,1.0,99000.0,NaN,0.0,30,[Fe/H],1.0,0.0
4,24 Sex c,24 Sex,Radial Velocity,0.0,86000.0,NaN,0.0,30,[Fe/H],1.0,0.0


**Вторая часть**

На самом деле для использования критерия нам нужны только значения из трех колонок таблицы: hostname, If planet mass>0.3 1; else 0 и If metal>0 1; else 0. Таблицу сопряженности для критерия хи-квадрат мы будем строить именно по ним. Проблема в том, что у одной звезды может быть несколько планет, поэтому одной звезде может соответствовать несколько строчек. Поскольку нас не интересует, сколько планет-гигантов у звезды, мы ограничимся проверкой наличия планеты гиганта у звезды и тем самым сократим таблицу: в новой таблице одной звезде будет соотетствовать только одна строчка, а также в таблице будет три столбца: название звезды, наличие/отсуствие планеты гиганта и металличность.

Посмотрим, сколько уникальных значений в столбце с названиями звёзд:

In [30]:
len(data['hostname'].value_counts())

133

Выделим металличность и наличие планеты-гиганта для каждой звезды:

In [31]:
data['hostname'].value_counts().index

Index(['Kepler-20', 'Kepler-62', 'Kepler-48', '61 Vir', 'Kepler-19', '47 UMa',
       'Kepler-289', 'Kepler-18', 'Kepler-30', 'Kepler-37',
       ...
       'CoRoT-16', 'BD+49 828', '51 Peg', 'CoRoT-22', 'BD+48 738', 'CoRoT-28',
       'CoRoT-20', 'Kepler-14', 'Kepler-1514', 'Kepler-75'],
      dtype='object', length=133)

In [32]:
metal_arr = np.zeros(len(data['hostname'].value_counts().index))
for i in range(len(metal_arr)):
    metal_arr[i] = np.array(data[data['hostname'] == data['hostname'].value_counts().index[i]]['If metal>0 1; else 0'])[0]
metal_arr

array([1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1.,
       0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1.,
       0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1.])

In [33]:
giant_ex = np.zeros(len(data['hostname'].value_counts().index))
for i in range(len(giant_ex)):
    cumsum = np.array(data[data['hostname'] == data['hostname'].value_counts().index[i]]['If planet mass>0.3 1; else'].cumsum())
    val = cumsum[len(cumsum)-1]
    if(val>0):
        giant_ex[i] = 1
    else:
        giant_ex[i] = 0

Значит осталось создать таблицу с 133 строками и тремя столбцами: название звезды, металличность, наличие планеты-гиганта.

In [35]:
table = {'hostname' : data['hostname'].value_counts().index,
         'If metal>0 1; else 0': metal_arr,
         'Наличие планеты-гиганта': giant_ex}
frame = pd.DataFrame(table)
frame.head()

,hostname,If metal>0 1; else 0,Наличие планеты-гиганта
0,Kepler-20,1.0,0.0
1,Kepler-62,0.0,0.0
2,Kepler-48,1.0,1.0
3,61 Vir,0.0,0.0
4,Kepler-19,0.0,1.0


**Третья часть**

разобьём данные на 4 корзины и составим таблицу сопряженности. По столбцам будет металличность звезды (первый -- больше 0, второй -- меньше). По строкам -- наличие планеты-гиганта.

In [36]:
table = np.histogram2d(frame['If metal>0 1; else 0'].to_numpy(), 
                                        frame['Наличие планеты-гиганта'].to_numpy(), 
                                        bins = [2,2])[0]
table

array([[19., 41.],
       [14., 59.]])

Условия применимости критерия хи-квадрат: 

1. сумма по всей таблице (n) > 40
2. произведение суммы по строчкам и суммы по столбцам, деленное на сумму по всей таблице меньше пяти не более, чем в пяти процента случаев

In [37]:
n = table.sum()
n

133.0

Как видим, пункт первый выполняется.

In [38]:
sum1 = table.sum(axis=0)
sum1

array([ 33., 100.])

In [39]:
sum2 = table.sum(axis=1)
sum2

array([60., 73.])

In [40]:
sum3 = np.arange(4).reshape(2, 2)
for i in range(2):
    for j in range(2):
        sum3[i][j] = sum1[i]*sum2[j]/n
sum3

array([[14, 18],
       [45, 54]])

Пункт два тоже. Значит можно применять критерий.

In [41]:
chi2, p_val = sps.chi2_contingency(table)[:2]
p_val

0.14496302274052134

**Вывод**

p-value большой, а значит отвержения гипотезы не происходит, то есть, если говорить в терминах этой задачи, связь между металличностью и наличием планеты-гиганта на данной выборке обнаружена не была.